## Load environment and libraries
This cell loads environment variables from `sample_gemini.env` and imports libraries used later. If you don't have `python-dotenv`, `requests`, `google-generativeai`, or `openai` installed, run the following command in your environment:
```
pip install python-dotenv requests google-generativeai openai
```

In [22]:
from dotenv import load_dotenv
import os
import requests
import json

# Load the env file shipped with this notebook (edit it with your key & model)
load_dotenv('sample_gemini.env')
GEMINI_API_KEY = os.getenv('GEMINI_API_KEY')
GEMINI_MODEL = os.getenv('GEMINI_MODEL', 'text-bison-001')  # Default to free-tier model

print('GEMINI_API_KEY present:', bool(GEMINI_API_KEY))
print('Using model:', GEMINI_MODEL)

GEMINI_API_KEY present: True
Using model: openrouter/sherlock-dash-alpha


## Low-dependency Gemini caller
This cell defines `call_gemini(prompt)` which first tries to use the `google.generativeai` Python package if available and falls back to a simple REST `requests` call to the Generative Language endpoint. The function returns plain text output when possible.

In [23]:
import json

def call_gemini(prompt, model=GEMINI_MODEL, api_key=GEMINI_API_KEY, max_tokens=512):
    """Call OpenRouter API using openai-python client."""
    if not api_key:
        raise ValueError('GEMINI_API_KEY is not set. Add it to sample_gemini.env')

    from openai import OpenAI

    client = OpenAI(
        base_url="https://openrouter.ai/api/v1",
        api_key=api_key,
    )

    completion = client.chat.completions.create(
        extra_headers={
            "HTTP-Referer": "https://example.com/",  # Optional
            "X-Title": "Example"  # Optional
        },
        extra_body={},
        model=model,
        messages=[
            {
                "role": "user",
                "content": [
                    {
                        "type": "text",
                        "text": prompt
                    }
                ]
            }
        ]
    )
    return completion.choices[0].message.content

# Example usage
response = call_gemini("What is Python?")
print("Response Content:", response)


Response Content: Python is a high-level, interpreted programming language known for its simplicity, readability, and versatility. Created by Guido van Rossum and first released in 1991 (named after the British comedy series *Monty Python*), it's one of the most popular languages today, used everywhere from web development to AI and data science.

### Key Features
- **Easy to Learn**: Uses clean, English-like syntax with indentation for code blocks (no curly braces needed).
- **Dynamically Typed**: No need to declare variable types upfront.
- **Interpreted**: Runs line-by-line via an interpreter, great for quick prototyping.
- **Cross-Platform**: Works on Windows, macOS, Linux, and more.
- **Massive Ecosystem**: Libraries like NumPy (math), Pandas (data), Django/Flask (web), TensorFlow/PyTorch (ML), and thousands more via pip.

### Common Uses
| Domain          | Examples/Tools                  |
|-----------------|---------------------------------|
| **Web Dev**     | Django, Flask, F

## Roadmap generator function
This cell defines `generate_roadmap(topic, level, duration)` which assembles a clear prompt for the model asking for a compact, actionable learning path. The return is the model's plain-text response.

In [ ]:
def generate_roadmap(topic, level='beginner', duration='8 weeks'):
    prompt = f'''You are an expert learning designer.
Create a concise learning roadmap for the topic: {topic}.
Learner level: {level}.
Suggested total duration: {duration}.
Output a short numbered list of 6-10 modules. For each module include: a one-line goal, suggested resources (URLs or short names), and suggested time. Keep it concise and actionable.
Provide the result as plain text.'''
    return call_gemini(prompt)

## Example usage
Run this cell after you add your API key to `sample_gemini.env`. Change the `topic`, `level`, and `duration` as you like. The notebook will print the model's roadmap output.

In [ ]:
# Example inputs (edit these)
topic = 'Python for Data Science'
level = 'beginner'
duration = '8 weeks'

print('Generating roadmap for:', topic)
roadmap = generate_roadmap(topic, level, duration)
print('\n--- ROADMAP ---\n')
print(roadmap)

## Notes & next steps
- Edit `sample_gemini.env` and set `GEMINI_API_KEY` to your credential and `GEMINI_MODEL` to the model name you want to use (e.g., `gemini-pro-1`).
- Authentication: you can provide either an API key (recommended for quick tests) or an OAuth access token (a short-lived bearer token):
  - API key: create an API key in Google Cloud Console, enable the Generative Language API and billing for your project, then paste the key into `sample_gemini.env`. The notebook will send it as a `?key=` query parameter.
  - OAuth token: if you prefer OAuth, obtain an access token (for example with `gcloud auth application-default print-access-token`) and paste that token into `GEMINI_API_KEY`; the notebook will send it as a Bearer token.
- Common reasons for a 401 error:
  - The Generative Language API is not enabled for the project.
  - Billing is not enabled on the project.
  - You're using the wrong credential type (e.g., pasting a service-account JSON instead of a token or API key).
- If you have `gcloud` installed, you can quickly get an AD token with this command in PowerShell:
```
gcloud auth application-default print-access-token
```
Copy the output and paste it into `sample_gemini.env` as `GEMINI_API_KEY=` (this token expires, so regenerate when needed).
- If you'd like, I can add a small cell to automatically fetch an access token using a service-account JSON or add a parser to structure model output into a DataFrame.

# Step 1: Roadmap Maker Workflow & Node Definitions

This notebook will implement the Roadmap/Learning Path Maker agent step by step. We'll start by defining the workflow and the main nodes:

**Workflow Nodes:**
1. Topic & Goal Selection
2. Background Assessment
3. Roadmap Generation
4. User Review & Feedback
5. Adaptive Branching
6. Final Output

Each node will be implemented as a function or cell, and we'll use a Python dictionary to manage state (user inputs, roadmap versions, feedback, etc.).

Let's begin with the first node: Topic & Goal Selection.

In [4]:
# Node 1: Topic & Goal Selection
# This cell collects the topic and learning goal from the user and stores them in the state dictionary.

state = {}

print("Welcome to the Roadmap/Learning Path Maker!")
topic = input("Enter the topic you want to learn (e.g., Python for Data Science): ")
goal = input("Describe your learning goal (e.g., Get a job, Build a project, Pass a course): ")

state['topic'] = topic
state['goal'] = goal

print(f"Selected Topic: {topic}")
print(f"Learning Goal: {goal}")

Welcome to the Roadmap/Learning Path Maker!
Selected Topic: Python for Data Science
Learning Goal: Build a project
Selected Topic: Python for Data Science
Learning Goal: Build a project


In [5]:
# Node 2: Background Assessment
# This cell collects information about your prior experience, preferred learning style, and available time.

experience = input("Briefly describe your prior experience with this topic (e.g., None, Some basics, Intermediate, Advanced): ")
learning_style = input("Preferred learning style (e.g., Videos, Reading, Projects, Interactive): ")
available_time = input("How much time can you dedicate per week? (e.g., 2 hours, 5 hours, 10 hours): ")

state['experience'] = experience
state['learning_style'] = learning_style
state['available_time'] = available_time

print("\nBackground Assessment Complete:")
print(f"Experience: {experience}")
print(f"Learning Style: {learning_style}")
print(f"Available Time: {available_time}")


Background Assessment Complete:
Experience: Some basics
Learning Style: projects
Available Time: 10


In [ ]:
# Node 3: Roadmap Generation
# This cell uses the collected state to generate a personalized learning roadmap using the LLM.

# Compose a prompt for the LLM based on user inputs
roadmap_prompt = f"""
You are an expert learning designer.
Create a concise, actionable learning roadmap for the topic: {state['topic']}.
Goal: {state['goal']}.
Learner background: {state['experience']}.
Preferred learning style: {state['learning_style']}.
Available time per week: {state['available_time']}.
Output a short numbered list of 6-10 modules. For each module include: a one-line goal, suggested resources (URLs or short names), and suggested time. Keep it concise and actionable.
Provide the result as plain text.
"""

roadmap = call_gemini(roadmap_prompt)
state['roadmap'] = roadmap

print("\n--- ROADMAP ---\n")
print(roadmap)

In [ ]:
# Node 4: User Review & Feedback
# This cell allows the user to review the generated roadmap and provide feedback for changes.

print("\nPlease review your generated roadmap above.")
feedback = input("Would you like to make any changes? (e.g., add/remove modules, adjust difficulty, change resources) If yes, describe your request. If no, type 'no': ")

state['feedback'] = feedback

if feedback.strip().lower() == 'no':
    print("Roadmap accepted. Proceeding to final output...")
else:
    print(f"Feedback received: {feedback}\nProceeding to adaptive branching...")

In [ ]:
# Node 5: Adaptive Branching
# This cell updates the roadmap based on user feedback, if any, and stores the revised version.

if state['feedback'].strip().lower() == 'no':
    revised_roadmap = state['roadmap']
else:
    # Compose a new prompt for the LLM including the feedback
    adaptive_prompt = f"""
    You are an expert learning designer.
    The user has requested the following changes to their roadmap: {state['feedback']}.
    Please revise the previous roadmap for the topic: {state['topic']}.
    Goal: {state['goal']}.
    Learner background: {state['experience']}.
    Preferred learning style: {state['learning_style']}.
    Available time per week: {state['available_time']}.
    Output a short numbered list of 6-10 modules. For each module include: a one-line goal, suggested resources (URLs or short names), and suggested time. Keep it concise and actionable.
    Provide the result as plain text.
    """
    revised_roadmap = call_gemini(adaptive_prompt)
    print("\n--- REVISED ROADMAP ---\n")
    print(revised_roadmap)

state['revised_roadmap'] = revised_roadmap

In [ ]:
# Node 6: Final Output
# This cell presents the final roadmap and summarizes user inputs and decisions.

print("\n===== FINAL ROADMAP =====\n")
print(state['revised_roadmap'])

print("\nSummary of your learning path setup:")
print(f"Topic: {state['topic']}")
print(f"Goal: {state['goal']}")
print(f"Experience: {state['experience']}")
print(f"Learning Style: {state['learning_style']}")
print(f"Available Time: {state['available_time']}")
if state['feedback'].strip().lower() == 'no':
    print("No changes requested to the initial roadmap.")
else:
    print(f"Feedback/changes requested: {state['feedback']}")
print("\nThank you for using the Roadmap/Learning Path Maker!")

# LangGraph Integration: Node Definitions

We'll now refactor the workflow to use LangGraph. Each step will be a node function, and the graph will manage state and transitions.

First, ensure you have LangGraph installed:
```
pip install langgraph
```

Next, we'll import LangGraph and define the node functions for each step.

In [24]:
# Import LangGraph and define node functions
from langgraph.graph import StateGraph

# Define state structure
class RoadmapState:
    def __init__(self):
        self.topic = None
        self.goal = None
        self.experience = None
        self.learning_style = None
        self.available_time = None
        self.roadmap = None
        self.feedback = None
        self.revised_roadmap = None

# Node 1: Topic & Goal Selection
def node_topic_goal(state: RoadmapState):
    print("Welcome to the Roadmap/Learning Path Maker!")
    state.topic = input("Enter the topic you want to learn (e.g., Python for Data Science): ")
    state.goal = input("Describe your learning goal (e.g., Get a job, Build a project, Pass a course): ")
    return state

# Node 2b: Topic Knowledge Assessment (LLM-generated questions)
def node_topic_assessment(state: RoadmapState):
    # Generate 6 quick assessment questions using the LLM
    prompt = f"""
You are an expert in {state.topic}.
Generate 6 short, practical yes/no questions to quickly assess a learner's knowledge of {state.topic}.
Each question should be phrased as:
Q: [question text]
A. Yes    B. No
Example: Q: Do you know about list comprehensions (e.g., [x for x in range(5)])? A. Yes    B. No
Return only the questions, numbered 1-6, in plain text.
"""
    questions_text = call_gemini(prompt)
    print("\n--- QUICK TOPIC ASSESSMENT ---\n")
    print(questions_text)
    # Parse questions (simple split, assuming numbered list)
    questions = [q.strip() for q in questions_text.split('\n') if q.strip() and q[0].isdigit()]
    answers = []
    for q in questions:
        print(q)
        ans = input("Your answer (A/B): ").strip().upper()
        answers.append(ans)
    # Estimate level: count Yes answers
    yes_count = sum(1 for a in answers if a == 'A')
    if yes_count <= 2:
        level = 'beginner'
    elif yes_count <= 4:
        level = 'intermediate'
    else:
        level = 'advanced'
    print(f"\nEstimated knowledge level: {level} ({yes_count} Yes out of 6)")
    state.assessment_questions = questions
    state.assessment_answers = answers
    state.estimated_level = level
    return state

# Node 3: Roadmap Generation
def node_generate_roadmap(state: RoadmapState):
    roadmap_prompt = f"""
You are an expert learning designer.
Create a concise, actionable learning roadmap for the topic: {state.topic}.
Goal: {state.goal}.
Learner background: {state.experience}.
Preferred learning style: {state.learning_style}.
Available time per week: {state.available_time}.
Output a short numbered list of 6-10 modules. For each module include: a one-line goal, suggested resources (URLs or short names), and suggested time. Keep it concise and actionable.
Provide the result as plain text.
"""
    state.roadmap = call_gemini(roadmap_prompt)
    print("\n--- ROADMAP ---\n")
    print(state.roadmap)
    return state

# Node 4: User Review & Feedback
def node_review_feedback(state: RoadmapState):
    print("\nPlease review your generated roadmap above.")
    state.feedback = input("Would you like to make any changes? (e.g., add/remove modules, adjust difficulty, change resources) If yes, describe your request. If no, type 'no': ")
    return state

# Node 5: Adaptive Branching
def node_adaptive_branch(state: RoadmapState):
    if state.feedback.strip().lower() == 'no':
        state.revised_roadmap = state.roadmap
    else:
        adaptive_prompt = f"""
You are an expert learning designer.
The user has requested the following changes to their roadmap: {state.feedback}.
Please revise the previous roadmap for the topic: {state.topic}.
Goal: {state.goal}.
Learner background: {state.experience}.
Preferred learning style: {state.learning_style}.
Available time per week: {state.available_time}.
Output a short numbered list of 6-10 modules. For each module include: a one-line goal, suggested resources (URLs or short names), and suggested time. Keep it concise and actionable.
Provide the result as plain text.
"""
        state.revised_roadmap = call_gemini(adaptive_prompt)
        print("\n--- REVISED ROADMAP ---\n")
        print(state.revised_roadmap)
    return state

# Node 6: Final Output
def node_final_output(state: RoadmapState):
    print("\n===== FINAL ROADMAP =====\n")
    print(state.revised_roadmap)
    print("\nSummary of your learning path setup:")
    print(f"Topic: {state.topic}")
    print(f"Goal: {state.goal}")
    print(f"Experience: {state.experience}")
    print(f"Learning Style: {state.learning_style}")
    print(f"Available Time: {state.available_time}")
    if state.feedback.strip().lower() == 'no':
        print("No changes requested to the initial roadmap.")
    else:
        print(f"Feedback/changes requested: {state.feedback}")
    print("\nThank you for using the Roadmap/Learning Path Maker!")
    return state

In [25]:
# Run the Roadmap/Learning Path workflow using node functions
state = RoadmapState()
state = node_topic_goal(state)
state = node_topic_assessment(state)  # Assessment node
state = node_generate_roadmap(state)
state = node_review_feedback(state)
state = node_adaptive_branch(state)
state = node_final_output(state)

Welcome to the Roadmap/Learning Path Maker!

--- QUICK TOPIC ASSESSMENT ---

1. Q: Do you know how to use virtual functions and polymorphism for entity inheritance in games?  
A. Yes    B. No

2. Q: Do you know about double buffering or swap chains for smooth rendering?  
A. Yes    B. No

3. Q: Do you know how to implement fixed timestep updates for consistent game physics?  
A. Yes    B. No

4. Q: Do you know std::unique_ptr and std::shared_ptr for managing game object memory?  
A. Yes    B. No

5. Q: Do you know how to use move semantics to avoid copying large game assets like meshes?  
A. Yes    B. No

6. Q: Do you know multithreading with std::thread for parallelizing game systems like AI?  
A. Yes    B. No
1. Q: Do you know how to use virtual functions and polymorphism for entity inheritance in games?

--- QUICK TOPIC ASSESSMENT ---

1. Q: Do you know how to use virtual functions and polymorphism for entity inheritance in games?  
A. Yes    B. No

2. Q: Do you know about double bu